In [28]:
import os
import zipfile
import numpy as np
import tifffile as tiff
import cv2
from PIL import Image

def process_tif(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for root, _, files in os.walk(input_folder):
        subfolder_name = os.path.basename(root)
        output_path = os.path.join(output_folder, f"{subfolder_name}.png")
        if os.path.exists(output_path):
            print(f"Output for {subfolder_name} already exists. Skipping.")
            continue
        
        for file in files:
            if file.endswith(".zip"):
                zip_path = os.path.join(root, file)
                try:
                    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                        zip_ref.extractall(root)
                except zipfile.BadZipFile:
                    print(f"Error: Cannot read zip file {zip_path}. Skipping.")
                    continue
                except Exception as e:
                    print(f"Error: {e} while extracting {zip_path}. Skipping.")
                    continue
                
                tif_files = [f for f in os.listdir(root) if f.endswith(".tif")]
                for tif_file in tif_files:
                    tif_path = os.path.join(root, tif_file)
                    with tiff.TiffFile(tif_path) as tif:
                        data = tif.asarray()
                    
                    # Assume data shape: (z, c, y, x)
                    z_slices, channels, _, _ = data.shape
                    montage_slices = data[::3]
                    num_slices = len(montage_slices)
                    n_rows = 10
                    n_cols = (num_slices + n_rows - 1) // n_rows
                    
                    # Create montage
                    montage_height = montage_slices.shape[2]
                    montage_width = montage_slices.shape[3]
                    montage_image = np.zeros((n_rows * montage_height, n_cols * montage_width, 3), dtype=np.uint8)
                    
                    for i, slice_data in enumerate(montage_slices):
                        
                        Tub_norm = cv2.normalize(slice_data[0], None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
                        Tub_cyan = cv2.merge((np.zeros_like(Tub_norm), Tub_norm, Tub_norm)) # merges R = 0, G and B to yield cyan

                        TubMask_norm = cv2.normalize(slice_data[3], None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
                        TubMask_cyan = cv2.merge((np.zeros_like(TubMask_norm), TubMask_norm, TubMask_norm)) # merges R = 0, G and B to yield cyan

                        DNA_norm = cv2.normalize(slice_data[1], None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
                        DNA_yellow = cv2.merge((DNA_norm, DNA_norm, np.zeros_like(DNA_norm))) # merges R, G and B = 0 to yield yellow

                        DNAMask_norm = cv2.normalize(slice_data[2], None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
                        DNAMask_yellow = cv2.merge((DNAMask_norm, DNAMask_norm, np.zeros_like(DNAMask_norm))) # merges R, G and B = 0 to yield yellow
                        
                        Pole_norm = cv2.normalize(slice_data[4], None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
                        Pole_grayscale = cv2.merge((Pole_norm, Pole_norm, Pole_norm))
                        
                        Tub_merged = np.maximum(Tub_cyan, cv2.addWeighted(TubMask_cyan, 0.75, np.zeros_like(TubMask_cyan), 0.25, 0))
                        DNA_merged = np.maximum(DNA_yellow, cv2.addWeighted(DNAMask_yellow, 0.75, np.zeros_like(DNAMask_yellow), 0.25, 0))
                        pre_merged = np.maximum(Tub_merged, DNA_merged)
                        merged = np.maximum(pre_merged, Pole_grayscale)
                        
                        row = (i % n_rows) * montage_height
                        col = (i // n_rows) * montage_width
                        montage_image[row:row + montage_height, col:col + montage_width] = merged
                    
                    # Save the montage
                    output_path = os.path.join(output_folder, f"{subfolder_name}.png")
                    Image.fromarray(montage_image).save(output_path)
                    print(f"Saved: {output_path}")

if __name__ == "__main__":
    input_folder = "/Volumes/arxivBeta/_Tobias/Opera/20250314/04_Spindle3D"
    output_folder = "/Volumes/arxivBeta/_Tobias/Opera/20250314/05_Spindle3D_QC_Juptyer"
    process_tif(input_folder, output_folder)

Output for ID0080_TK_CID_RNAiScreen_63x_PreciScan_250314 - R07-C03-F17_50000_IM_KIF11_orig_obj1_crop already exists. Skipping.
Output for ID0080_TK_CID_RNAiScreen_63x_PreciScan_250314 - R07-C03-F18_50000_IM_KIF11_orig_obj1_crop already exists. Skipping.
Output for ID0080_TK_CID_RNAiScreen_63x_PreciScan_250314 - R07-C03-F19_50000_IM_KIF11_orig_obj1_crop already exists. Skipping.
Output for ID0080_TK_CID_RNAiScreen_63x_PreciScan_250314 - R07-C03-F1_50000_IM_KIF11_orig_obj1_crop already exists. Skipping.
Output for ID0080_TK_CID_RNAiScreen_63x_PreciScan_250314 - R07-C03-F20_50000_IM_KIF11_orig_obj1_crop already exists. Skipping.
Output for ID0080_TK_CID_RNAiScreen_63x_PreciScan_250314 - R07-C03-F21_50000_IM_KIF11_orig_obj1_crop already exists. Skipping.
Output for ID0080_TK_CID_RNAiScreen_63x_PreciScan_250314 - R07-C03-F22_50000_IM_KIF11_orig_obj1_crop already exists. Skipping.
Output for ID0080_TK_CID_RNAiScreen_63x_PreciScan_250314 - R07-C03-F24_50000_IM_KIF11_orig_obj1_crop already exi